<a href="https://colab.research.google.com/github/juhee3199/AI-project/blob/master/%EC%8B%A4%EC%8A%B5%20code/%20MNIST_CNN_LeNet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.9
training_epochs = 50
batch_size = 256

- Lenet 5의 입력이 32*32 이기 때문에 resize가 필요하다.

In [ ]:
# TorchVision의 MNIST 영상이 28x28로 가공되어 있어, Original 입력 사이즈 32x32로 맞추기 위해
# import torchvision.transforms as transforms

transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

In [ ]:

# MNIST dataset
# transform이 mnist_train & mnist_test에 반영되는게 아이라 train_loader & test_loader에서 반영되는 듯
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms,
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms,
                         download=True)



In [ ]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


- LeNet5의 discription을 보고 아래 코드 설계 (15주차 10p)


In [ ]:
# sigmoid 사용 ver

class LeNet5(torch.nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__() 
                                                                      #32*32
    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)   #28*28
    self.x1 = torch.nn.Sigmoid()
    #self.x1 = torch.nn.Tanh()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)              #14*14

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)  #10*10
    self.x3 = torch.nn.Sigmoid()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2)              #5*5

# 3개의 fully connected layer ( C5, F6, output)

    #self.c5 = torch.nn.Conv2d(16,120,kernel_size=5,stride=1,padding=0)
    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Sigmoid()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Sigmoid()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        

    out = out.view(out.size(0),-1) # 1D로 변환. batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [ ]:
# tanh 사용 ver

class LeNet5_(torch.nn.Module):
  def __init__(self):
    super(LeNet5_,self).__init__() 

    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
    self.x1 = torch.nn.Tanh()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
    self.x3 = torch.nn.Tanh()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2) 

    self.fl = torch.nn.Faltten()

    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Tanh()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Tanh()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        
    out = out.view(out.size(0),-1) # batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [ ]:
model = LeNet5().to(device)

In [ ]:

criterion = torch.nn.CrossEntropyLoss().to(device)   
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [ ]:

#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:  
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 2.31528282
[Epoch:    2] cost = 2.30350852
[Epoch:    3] cost = 2.30266261
[Epoch:    4] cost = 2.30256438
[Epoch:    5] cost = 2.30229187
[Epoch:    6] cost = 2.30218005
[Epoch:    7] cost = 2.30205131
[Epoch:    8] cost = 2.30189776
[Epoch:    9] cost = 2.30148029
[Epoch:   10] cost = 2.30139041
[Epoch:   11] cost = 2.30023575
[Epoch:   12] cost = 2.27036929
[Epoch:   13] cost = 1.80259049
[Epoch:   14] cost = 0.801855922
[Epoch:   15] cost = 0.441283524
[Epoch:   16] cost = 0.291419446
[Epoch:   17] cost = 0.221858129
[Epoch:   18] cost = 0.175194353
[Epoch:   19] cost = 0.15169993
[Epoch:   20] cost = 0.128243178
[Epoch:   21] cost = 0.114277944
[Epoch:   22] cost = 0.0995416418
[Epoch:   23] cost = 0.091726467
[Epoch:   24] cost = 0.0843576193
[Epoch:   25] cost = 0.0774186701
[Epoch:   26] cost = 0.0743131265
[Epoch:   27] cost = 0.0683426559
[Epoch:   28] cost = 0.0619802848
[Epoch:   29] cost = 0.0595374145
[Epoch:   30]

In [ ]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
  model.eval()

  for X, Y in test_loader:   # minibatch를 사용해서 평가
    Y = Y.to(device)
    X = X.to(device)


    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)



Accuracy: 0.9885817307692307


Accuracy: 0.8358998107910156